## Feature backfill


Imports

In [1]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
from functions import util
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# with open('../data/keys/hopsworks-api-key.txt', 'r') as file:
#     os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

# logout from hopsworks first
project = hopsworks.login(project="ML_Project_Electricity", api_key_value=os.environ["HOPSWORKS_API_KEY"])
print(f"Project: {project.name}")

# util.purge_project(proj)


x5PZCK0patagTBDP.SjxqfFY5JlMcD4HCF4BEd0rDdMJVabM44KJEdZRIVrYSoxvGd8lfgz23KgblRcI9
2025-01-08 14:08:26,669 INFO: Initializing external client
2025-01-08 14:08:26,670 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-08 14:08:26,670 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-08 14:08:27,805 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207495
Project: ML_Project_Electricity


## LOAD ALL THE DATA

In [3]:
# Start by loading historical electricity prices:
# Load the Elspotprices.csv file
csv_file = Path("../data/Elspotprices.csv")
e_df = pd.read_csv(csv_file, delimiter=";")
e_df.head()

# Remove HourUTC and SpotpriceDKK column
e_df = e_df.drop(columns=['HourUTC', 'SpotPriceDKK'])

# Rename HourDK to Time and make it proper datetime format
e_df = e_df.rename(columns={"HourDK": "time"})
e_df['time'] = pd.to_datetime(e_df['time'])

# Make the Time column the index of the DataFrame
# e_df = e_df.set_index('time')

# Format SpotPriceEUR into a float
e_df['SpotPriceEUR'] = e_df['SpotPriceEUR'].str.replace(',', '.').astype(float)

# Split the DataFrame based on the 'PriceArea' column
price_area_groups = e_df.groupby('PriceArea')

# Create separate DataFrames for each PriceArea
price_area_dfs = {price_area: group for price_area, group in price_area_groups}

# Access individual DataFrames
se3_df = price_area_dfs['SE3']
se4_df = price_area_dfs['SE4']

se3_df.head()

,time,PriceArea,SpotPriceEUR
0,2024-12-31 23:00:00,SE3,2.24
3,2024-12-31 22:00:00,SE3,7.93
4,2024-12-31 21:00:00,SE3,14.98
7,2024-12-31 20:00:00,SE3,21.51
8,2024-12-31 19:00:00,SE3,23.01


In [ ]:
# add a rolling mean of the last 24*4 hours to the price data
# Sort the data by time before doing the rolling mean
se3_df = se3_df.sort_values('time')
se4_df = se4_df.sort_values('time')

se3_df['spot_price_rolling'] = se3_df['SpotPriceEUR'].rolling(window=24*7).mean()
se4_df['spot_price_rolling'] = se4_df['SpotPriceEUR'].rolling(window=24*7).mean()

# Add a rolling mean of the last 3 hours
se3_df['spot_price_rolling_3h'] = se3_df['SpotPriceEUR'].rolling(window=3).mean()
se4_df['spot_price_rolling_3h'] = se4_df['SpotPriceEUR'].rolling(window=3).mean()

se4_df.head(3)

,time,PriceArea,SpotPriceEUR,spot_price_rolling,spot_price_rolling_3h
35087,2023-01-01 00:00:00,SE4,2.01,NaN,NaN
35085,2023-01-01 01:00:00,SE4,1.38,NaN,NaN
35083,2023-01-01 02:00:00,SE4,0.09,NaN,1.16


In [7]:
# Load the weather data for Stockholm and Malmo

weather_sthlm_df = pd.read_csv("../data/Hourly_weather_stockholm.csv", header=2)
weather_malmo_df = pd.read_csv("../data/Hourly_weather_malmo.csv", header=2)

# weather_sthlm_df.head()

# Splite the 'time' column into 'Date', 'Weekday', 'Month' and 'Hour' columns
weather_sthlm_df['time'] = pd.to_datetime(weather_sthlm_df['time'])
weather_sthlm_df['Date'] = weather_sthlm_df['time'].dt.date
weather_sthlm_df['Weekday'] = weather_sthlm_df['time'].dt.weekday
weather_sthlm_df['Month'] = weather_sthlm_df['time'].dt.month
weather_sthlm_df['Hour'] = weather_sthlm_df['time'].dt.hour

weather_malmo_df['time'] = pd.to_datetime(weather_malmo_df['time'])
weather_malmo_df['Date'] = weather_malmo_df['time'].dt.date
weather_malmo_df['Weekday'] = weather_malmo_df['time'].dt.weekday
weather_malmo_df['Month'] = weather_malmo_df['time'].dt.month
weather_malmo_df['Hour'] = weather_malmo_df['time'].dt.hour

weather_sthlm_df.head()

,time,temperature,precipitation,cloud_cover,wind_speed_10m,Date,Weekday,Month,Hour
0,2023-01-01 00:00:00,2.8,0.0,87,15.3,2023-01-01,6,1,0
1,2023-01-01 01:00:00,2.4,0.0,100,14.8,2023-01-01,6,1,1
2,2023-01-01 02:00:00,1.6,0.0,100,11.2,2023-01-01,6,1,2
3,2023-01-01 03:00:00,0.8,0.0,100,10.4,2023-01-01,6,1,3
4,2023-01-01 04:00:00,0.2,0.0,100,9.7,2023-01-01,6,1,4


In [8]:
# Combine the hourly weather data with daily total sunshine duration data

# Load the daily sunshine duration data for SE3 (Stockholm)
sunshine_stockholm_df = pd.read_csv("../data/Daily_sunshine_stockholm.csv", header=2)
sunshine_stockholm_df['Date'] = pd.to_datetime(sunshine_stockholm_df['time'])
sunshine_stockholm_df = sunshine_stockholm_df.drop(columns=['time'])
sunshine_stockholm_df = sunshine_stockholm_df.set_index('Date')

# Merge the weather data (weather_sthlm_df) with the sunshine duration data (if it hasn't been done already)
if 'sunshine_duration' not in weather_sthlm_df.columns:
    # Merge the weather data with the sunshine duration data
    weather_sthlm_df['Date'] = weather_sthlm_df['time'].dt.date
    weather_sthlm_df = weather_sthlm_df.set_index('Date')
    weather_sthlm_df = weather_sthlm_df.join(sunshine_stockholm_df, how='left')
    weather_sthlm_df = weather_sthlm_df.reset_index()
    print("Merge completed.")
else:
    print("Merge has already been done.")

weather_sthlm_df.head()

# Load the daily sunshine duration data for SE4 (Malmö)
sunshine_malmo_df = pd.read_csv("../data/Daily_sunshine_malmo.csv", header=2)
sunshine_malmo_df['Date'] = pd.to_datetime(sunshine_malmo_df['time'])
sunshine_malmo_df = sunshine_malmo_df.drop(columns=['time'])
sunshine_malmo_df = sunshine_malmo_df.set_index('Date')

# sunshine_malmo_df.head()

# Merge the weather data with the sunshine duration data (if it hasn't been done already)
if 'sunshine_duration' not in weather_malmo_df.columns:
    # Merge the weather data with the sunshine duration data
    weather_malmo_df['Date'] = weather_malmo_df['time'].dt.date
    weather_malmo_df = weather_malmo_df.set_index('Date')
    weather_malmo_df = weather_malmo_df.join(sunshine_malmo_df, how='left')
    weather_malmo_df = weather_malmo_df.reset_index()
    print("Merge completed.")
else:
    print("Merge has already been done.")

weather_sthlm_df.head()

Merge completed.
Merge completed.


,Date,time,temperature,precipitation,cloud_cover,wind_speed_10m,Weekday,Month,Hour,sunshine_duration
0,2023-01-01,2023-01-01 00:00:00,2.8,0.0,87,15.3,6,1,0,13654.35
1,2023-01-01,2023-01-01 01:00:00,2.4,0.0,100,14.8,6,1,1,13654.35
2,2023-01-01,2023-01-01 02:00:00,1.6,0.0,100,11.2,6,1,2,13654.35
3,2023-01-01,2023-01-01 03:00:00,0.8,0.0,100,10.4,6,1,3,13654.35
4,2023-01-01,2023-01-01 04:00:00,0.2,0.0,100,9.7,6,1,4,13654.35


## DATA CLEANING

In [9]:
# Drop any rows with missing values
weather_sthlm_df = weather_sthlm_df.dropna()
weather_malmo_df = weather_malmo_df.dropna()

## Create feature groups

In [13]:
fs = project.get_feature_store()

# Create a feature group for the electricity prices in SE3
se3_fg = fs.get_or_create_feature_group(
    name="se3_electricity_prices", 
    version=1, 
    description="Electricity prices in SE3",
    primary_key=["time"],
    event_time="time")

# Create a feature group for the electricity prices in SE4
se4_fg = fs.get_or_create_feature_group(
    name="se4_electricity_prices", 
    version=1, 
    description="Electricity prices in SE4",
    primary_key=["time"],
    event_time="time")

# Create a feature group for the weather data in Stockholm
sthlm_fg = fs.get_or_create_feature_group(
    name="stockholm_weather", 
    version=1, 
    description="Weather data in Stockholm",
    primary_key=["time"],
    event_time="time")

# # Create a feature group for the weather data in Malmö
malmo_fg = fs.get_or_create_feature_group(
    name="malmo_weather", 
    version=1, 
    description="Weather data in Malmö",
    primary_key=["time"],
    event_time="time")



## Insert Historical data into feature groups

In [14]:
#Insert data into the feature groups
se3_fg.insert(se3_df)
se4_fg.insert(se4_df)
# sthlm_fg.insert(weather_sthlm_df)
# malmo_fg.insert(weather_malmo_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1207495/fs/1195127/fg/1393774


Uploading Dataframe: 100.00% |██████████| Rows 17544/17544 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: se3_electricity_prices_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207495/jobs/named/se3_electricity_prices_1_offline_fg_materialization/executions
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1207495/fs/1195127/fg/1393775


Uploading Dataframe: 100.00% |██████████| Rows 17544/17544 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: se4_electricity_prices_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207495/jobs/named/se4_electricity_prices_1_offline_fg_materialization/executions


(Job('se4_electricity_prices_1_offline_fg_materialization', 'SPARK'), None)